In [28]:
import sys
sys.path.append('../')
from tools.utils import rm_if_exist
from tools.img import rgb_to_yuv, yuv_to_rgb
import torch
import PIL
import numpy as np
import os
import pywt
from tools.img import tensor2ndarray, ndarray2tensor

In [29]:
import PIL.Image


img = PIL.Image.open('/home/chengyiqiu/code/INBA/data/imagenette2/train/n02102040/ILSVRC2012_val_00008334.JPEG')
img = img.resize((224, 224))
x_0 = np.array(img)
rm_if_exist('./franwork/')
os.makedirs('./franwork/')
PIL.Image.Image.save(img, f'./franwork/0_clean.jpeg')

Folder './franwork/' has been removed.


In [30]:
coeff = pywt.wavedec2(x_0, wavelet='haar', level=1, axes=(0, 1))
LL = coeff[0]
(LH, HL, HH) = coeff[-1]

# for show to clip
# LL = LL.clip(0, 255)
# HH = HH.clip(0, 255)
PIL.Image.Image.save(PIL.Image.fromarray(LL.astype(np.uint8)), f'./franwork/1_LL.jpeg')
PIL.Image.Image.save(PIL.Image.fromarray(HH.astype(np.uint8)), f'./franwork/2_HH.jpeg')

In [31]:
def inject_trigger(target, window_size, trigger_size, phase_trigger, ch_list=[1, 2], mode='train'):
    for ch in ch_list:
        for i in range(0, target.shape[0], window_size):
            for j in range(0, target.shape[1], window_size):
                tmp = target[i:i+window_size, j:j+window_size, ch]
                tmp_fft = np.fft.fft2(tmp, axes=(0, 1))
                amp, pha = np.abs(tmp_fft), np.angle(tmp_fft)
                PIL.Image.Image.save(PIL.Image.fromarray(pha.astype(np.uint8)), f'./franwork/5_pha.jpeg')
                pha[-1-trigger_size:-1, -1-trigger_size:-1] = phase_trigger
                PIL.Image.Image.save(PIL.Image.fromarray(pha.astype(np.uint8)), f'./franwork/6_pha_P.jpeg')
                tmp_fft = amp * np.exp(1j * pha)
                tmp = np.fft.ifft2(tmp_fft, axes=(0, 1))
                tmp = tmp.real
                target[i:i+window_size, j:j+window_size, ch] = tmp
    return target

window_size = 4
trigger_size = 2
phase_trigger = -np.pi
ch_list = [1]

HH_yuv = np.stack(rgb_to_yuv(HH[:,:,0], HH[:,:,1], HH[:,:,2]), axis=-1)
LL_yuv = np.stack(rgb_to_yuv(LL[:,:,0], LL[:,:,1], LL[:,:,2]), axis=-1)
PIL.Image.Image.save(PIL.Image.fromarray(LL_yuv[:, :, 1].astype(np.uint8)), f'./franwork/3_LL_U.jpeg')
PIL.Image.Image.save(PIL.Image.fromarray(HH_yuv[:, :, 1].astype(np.uint8)), f'./franwork/4_HH_U.jpeg')


HH_yuv = inject_trigger(HH_yuv, window_size, trigger_size, phase_trigger, ch_list)
LL_yuv = inject_trigger(LL_yuv, window_size, trigger_size, phase_trigger, ch_list)
HH = np.stack(yuv_to_rgb(HH_yuv[:,:,0], HH_yuv[:,:,1], HH_yuv[:,:,2]), axis=-1)
LL = np.stack(yuv_to_rgb(LL_yuv[:,:,0], LL_yuv[:,:,1], LL_yuv[:,:,2]), axis=-1) 

PIL.Image.Image.save(PIL.Image.fromarray(LL_yuv[:, :, 1].astype(np.uint8)), f'./franwork/7_LL_U_P.jpeg')
PIL.Image.Image.save(PIL.Image.fromarray(HH_yuv[:, :, 1].astype(np.uint8)), f'./franwork/8_HH_U_P.jpeg')


In [32]:
coeff[0] = LL
coeff[-1] = (LH, HL, HH)
x_p = pywt.waverec2(coeff, wavelet='haar', axes=(0, 1))
x_p = x_p.clip(0, 255)
PIL.Image.Image.save(PIL.Image.fromarray(x_p.astype(np.uint8)), f'./franwork/9_x_p.jpeg')

In [33]:
def zero_out_array(array, ratio):
    size = array.size
    num_zero = int(size * ratio)
    indices = np.random.permutation(size)[:num_zero]
    flat_array = array.flatten()
    flat_array[indices] = 0
    return flat_array.reshape(array.shape)
tg = x_p - x_0
PIL.Image.Image.save(PIL.Image.fromarray(tg.astype(np.uint8)), f'./franwork/10_tg.jpeg')
tg = zero_out_array(tg, 0.4)
x_p = x_0 + tg
PIL.Image.Image.save(PIL.Image.fromarray(tg.astype(np.uint8)), f'./franwork/11_tg_m.jpeg')
PIL.Image.Image.save(PIL.Image.fromarray(x_p.astype(np.uint8)), f'./franwork/12_x_p.jpeg')
print(type(x_p))

<class 'numpy.ndarray'>


In [34]:
u_noise = np.random.uniform(0, 255, size=x_p.shape)
x_p_enhanced = x_p + 0.05 * u_noise
PIL.Image.Image.save(PIL.Image.fromarray(u_noise.astype(np.uint8)), f'./franwork/13_uniform_noise.jpeg')
PIL.Image.Image.save(PIL.Image.fromarray(x_p_enhanced.astype(np.uint8)), f'./franwork/14_x_p_enhanced.jpeg')